In [44]:
from dotenv import dotenv_values
import pandas as pd
import requests
import json
# my twitter keys are stores in a file outside the git repository for security reasons
config = dotenv_values("/Users/ambrosedesmond/CCT_Projects/Ambrose_MSC_DS_CA2/.env")
bearer_token = config["BEARER_TOKEN"]
# this is the location of the twitter api access , looking for the most recent tweets
search_url = "https://api.twitter.com/2/tweets/search/recent"

In [55]:
# set up a dataframe to save tweets, only saving author and the tweet
# from https://www.geeksforgeeks.org/python-pandas-dataframe-append/?ref=gcse

tweet_df = pd.DataFrame({"author_id":[],
                   "text":[]})



# Set up the query we want to pass to api , is a json object
    #  -is = is not a retweet. i dont want retweets showing up in my data 
    # also pulling in the author id and name
query_params1 = {
    'query':'BREXIT farming -is:retweet',
    'max_results': '100',
    'tweet.fields':'author_id',
    'user.fields':'name',

}
query_params2 = {
    'query':'BREXIT farming -is:retweet',
    'max_results': '100',
    'tweet.fields':'author_id',
    'user.fields':'name',
    'next_token' : f'{next}',
}
def bearer_oauth(r):
    r.headers['Authorization'] = f"Bearer {bearer_token}"
    r.headers['User-Agent'] = "v2RecentSerchPython"
    return r

def connect_to_endpoint(url,params):
    """ Function to connect to twitter API"""
    response = requests.get(url,auth=bearer_oauth, params=params)
    #responce.status_code 200 is good anything else is an error
    #404 page not found
    return response.json()

def append_df():
    tweet_df.loc[len(tweet_df.index)] = [(json_response['data'][0]['author_id']),(json_response['data'][0]['text'])]
    #tweet_df.append(json_response['data'][0]['author_id'],json_response['data'][0]['text'])
    
    
def twit_call(iter_count):
    json_response = connect_to_endpoint(search_url, query_params1)
    parsedRes = json.dumps(json_response, indent=4, ensure_ascii=False)
    if json_response['meta']['next_token'] != None :
        for count in range(iter_count-1):
            json_response = connect_to_endpoint(search_url, query_params2)
            if json_response['meta']['next_token'] == None:
    # setting indent to 4 to make more readable , ensure_ascii will return all charecters when false
    # so we can get to see emojis in our twitter text
    
    
    #print(parsedRes)
# pagenation is taking the next_token ang getting the next batch of tweets
twit_call(5)

In [53]:
json_response['data'][0]['author_id']
json_response['data'][0]['text']
json_response['meta']['next_token']

'b26v89c19zqg8o3fqk123p8bg6p7b889l6j5sxow7yikd'

In [10]:
twitter_keys=f'''keys:
    access_token: {config["API_KEY"]}
    access_token_secret: {config["API_KEY_SECRET"]}
    bearer_token: {config["BEARER_TOKEN"]}
'''
with open("~/.twitter-keys.yaml","w") as file:
    file.write(twitter-keys)
with open("~/.twitter-keys.yaml", "w") as file:
    file.write(twitter_keys)


FileNotFoundError: [Errno 2] No such file or directory: '~/.twitter-keys.yaml'

In [24]:
from twitter_stream import FilteredStream

from time import time

start = time()
stream = FilteredStream()
rule = {
    "add": [
        {"value": '"World Cup" -is:retweet',
        "tag" : "soccer"
        }
    ]
}
stream.add_rule(data=rule)
tweetlist = []
for tweet in stream.connect():
    parsedTweet = json.dumps(tweet, indent=4, ensure_ascii=False)
    print(parsedTweet)
    print(f"There are {len(tweetlist)} tweets about {len(tweetlist)/(time()-start)} tweets per")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ambrosedesmond/.twitter-keys.yaml'